In [3]:
import logging
import requests
import time

from epdlib import Screen, Update
from epdlib import Layout

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
logging.basicConfig(level=logging.DEBUG)
logging.debug('foobar')

DEBUG:root:foobar


In [6]:
logging.getLogger().setLevel('INFO')

# PaperPi Structure
Supervisor will loop and poll each plugin for an update. Plugins are all`Plugin()` objects with a `poll` method

When polled `Plugin()` objects respond with a structured list (tuple/dict) with the following information:
* Priority(`int`): 0 (high) -- 10+ (low); plugins with negative values are excluded
* Update Required(`bool`): True update; False do not update

Plugin objects require the following parameters:
* configuration(`dict`) argparse/config.ini style dict
* resolution (this may be pulled from the configuration?)
* name(`str`) human readable name for logging and reference
* update_function(`func`): function that will run and provide updates and status

In [7]:
def strict_enforce(*types):
    """strictly enforce type compliance within classes
    
    Usage:
        @strict_enforce(type1, type2, (type3, type4))
        def foo(val1, val2, val3):
            ...
    """
    def decorator(f):
        def new_f(self, *args, **kwds):
            #we need to convert args into something mutable   
            newargs = []        
            for (a, t) in zip(args, types):
                if not isinstance(a, t):
                    raise TypeError(f'"{a}" is not type {t}')
#                 newargs.append( t(a)) #feel free to have more elaborated convertion
            return f(self, *args, **kwds)
        return new_f
    return decorator

In [8]:
class A:
    '''dummy class for testing'''
    pass



In [9]:
# borrowed from https://stackoverflow.com/a/31923812/5530152

class TooSoon(Exception):
    """Can't be called so soon"""
    pass

class CoolDownDecorator(object):
    def __init__(self,func,interval):
        self.func = func
        self.interval = interval
        self.last_run = 0
    def __get__(self,obj,objtype=None):
        if obj is None:
            return self.func
        return partial(self,obj)
    def __call__(self,*args,**kwargs):
        now = time.time()
        if now - self.last_run < self.interval:
            raise TooSoon("cooldown -- wait {0} seconds".format(self.last_run + self.interval - now))
        else:
            self.last_run = now
            return self.func(*args,**kwargs)


In [61]:
import time
import hashlib
class Plugin:
    '''base class for epaper_display plugins'''
    
    def __init__(self, name=None, resolution=None, config={}, layout=None,
                 update_function=None, max_priority=-1, refresh_rate=60, 
                 min_display_time=30):
        self.name = name
        self.update_hash = self._generate_hash()
        self.resolution = resolution
        self.config = config
        if self.resolution:
            self.layout = layout
        self.max_priority = max_priority
        self.priority = -1
        self.refresh_rate = refresh_rate
        self.min_display_time = min_display_time
        self.data = {}
        self._add_update_function(update_function)

    @property
    def name(self):
        '''name of plugin derived from configuration file -- always in the format: `Plugin: XXXXX`
        name(`str`): Plugin: Name'''
        return self._name

    @name.setter
    def name(self, name):
        self._name = str(name)

    @property
    def resolution(self):
        '''width x height of screen
        resolution(`tuple` of `int`): (W x H)'''
        return self._resolution
    
    @resolution.setter
    def resolution(self, resolution):
        if isinstance(resolution, (type(None), list, tuple)):
            self._resolution = resolution
            if hasattr(self, 'layout_obj'):
                # trigger recalculation of the layout if the resolution changes
                layout = self.layout
                self.layout = layout
        else:
            raise ValueError('`resolution` must be a list-like object')
        
    @property
    def layout(self):
        '''layout dictionary for placing text and images blocks on the screen'''
        return self.layout_obj.layout
    
    @layout.setter
    def layout(self, layout):
        if self.resolution:
            self.layout_obj = Layout(resolution=self.resolution, layout=layout) 
        else:
            raise ValueError(f'cannot set `layout` without a valid resolution: {self.resolution}')
                             
    @property
    def max_priority(self):
        '''maximum priority for displaying this module in the update loop
            positive values that approach zero are higher priority; zero is highest
            negative values are excluded entirely from update loop
        
        max_priority(`int` or `float`)
        '''
        return self._max_priority
    
    @max_priority.setter
    @strict_enforce(int, float)
    def max_priority(self, max_priority):
        self._max_priority = max_priority        
    
    @property
    def refresh_rate(self):
        '''time in seconds beteween refreshes
        refresh_rate(`int` or `float`): N'''
        return self._refresh_rate
    
    @refresh_rate.setter
    @strict_enforce(int, float)
    def refresh_rate(self, refresh_rate):
        if refresh_rate <= 0:
            raise ValueError('refresh_rate must be > 0')
        else:
            self._refresh_rate = refresh_rate

    @property
    def min_display_time(self):
        '''minimum time to display plugin in seconds
        min_display_time(`int` or `float`): N'''
        return self._min_display_time
    
    @min_display_time.setter
    @strict_enforce(int, float)
    def min_display_time(self, min_display_time):
        if min_display_time <= 0:
            raise ValueError('min_display_time must be > 0')
        else:
            self._min_display_time = min_display_time
    
    @property 
    def data(self):
        '''data returned from self.update_function
        data(`dict`): {'key': 'value'}'''
        return self._data
    
    @data.setter
    @strict_enforce(dict)
    def data(self, data):
        self._data = data    
    
    @property
    def image(self):
        '''constructed image based on the layout'''
        return self.layout_obj.concat()    
    
    def _generate_hash(self):
        my_hash = hashlib.sha1()
        my_hash.update(str(time.time()).encode('utf-8')+str(self.name).encode('utf-8'))
        return my_hash.hexdigest()[:10]
    
    def _null_function(self, *args, **kwargs):
        logging.info('default placeholder function')
        return (False, {}, -1)                             
    
    def _add_update_function(self, function):
        '''add an update_function to the namespace of this instance of object
            all update_functions must accept at a minimum the following kwargs:
                * config(`dict`) configuration dictionary passed to all update functions
            all update_functions must return a tuple of:
                (update, data, priority)
                
                update(`bool`): True, data has been updated; display requires update
                data(`dict`): structured for a Layout().update 
                priority(`int` or `float`): priority for inclusion in display loop
                    positive values that approach 0 have higher priority

        Args:
            fuction(`function`): function to add as self.update_function
            
        see https://stackoverflow.com/a/2982/5530152 regarding descriptor protocol''' 
        if function:
            self.update_function = function.__get__(self)
        else:
            self.update_function = self._null_function.__get__(self) 
            
    def _CoolDown(self):
        def applyDecorator(func):
            decorator = CoolDownDecorator(func=func,interval=self.refresh_rate)
            return wraps(func)(decorator)
        return applyDecorator


In [ ]:
# print(my_plugin.timer.last_updated)
# print(my_plugin.update_rate)
# my_plugin.poll()


In [ ]:
# section = 'Plugin: LMS'
# my_plugin = Plugin(name=section,
#                            resolution=config['main']['resolution'],
#                            layout=config[section]['layout'],
#                            update_function=config[section]['update_function'],
#                            config=config[section], 
#                            min_display_time=config[section]['min_display_time'],
#                            update_rate=config[section]['update_rate'])

In [30]:
clock_layout = {
    'digit_time': {
        'image': None,
        'max_lines': 3,
        'width': 1,
        'height': 1,
        'abs_coordinates': (0, 0),
        'rand': True,
        'font': '../fonts/Dosis/Dosis-VariableFontwght.ttf',
    },
}


test = A()
test.config = {'update_interval': 10}
test.timer = Update()
test.max_priority = 1

from datetime import datetime
def clock_plugin(self):
    data = {'digit_time': datetime.now().strftime("%H:%M:%S")}
    priority = self.max_priority
    is_updated = True
    
    return (is_updated, data, priority)

In [31]:
clock_plugin(test)

(True, {'digit_time': '18:40:08'}, 1)

In [75]:
slim_layout = {
    'title': {                       # text only block
        'image': None,               # do not expect an image
        'max_lines': 2,              # number of lines of text
        'width': 1,                  # 1/1 of the width - this stretches the entire width of the display
        'height': 2/3,               # 1/3 of the entire height
        'abs_coordinates': (0, 0),   # this block is the key block that all other blocks will be defined in terms of
        'hcenter': True,             # horizontally center text
        'vcenter': True,             # vertically center text 
        'relative': False,           # this block is not relative to any other. It has an ABSOLUTE position (0, 0)
        'font': '../fonts/Anton/Anton-Regular.ttf', # path to font file
        'font_size': None            # Calculate the font size because none was provided
    },

    'artist': {
        'image': None,
        'max_lines': 1,
        'width': 1,
        'height': 1/3,
        'abs_coordinates': [0, None],   # X = 0, Y will be calculated
        'hcenter': True,
        'vcenter': True,
        'font': '../fonts/Dosis/Dosis-VariableFontwght.ttf',
        'relative': ['artist', 'title'], # use the X postion from abs_coord from `artist` (this block: 0)
                                       # calculate the y position based on the size of `title` block
    }
}
slim_test = A()
slim_test.config = {'player_name': 'MacPlay', 'refresh_rate': 30}
slim_test.priority = -1
slim_test.max_priority = 0

import lmsquery
def slim_plugin(self):
    now_playing = None
    data = {
        'id': 0,
        'title': 'Err: No Player',
        'artist': 'Err: No Player',
        'coverid': 'Err: No Player',
        'duration': 0,
        'album_id': 'Err: No Player',
        'genre': 'Err: No Player',
        'album': 'Err: No Player',
        'artwork_url': 'Err: No Player',
        'mode': 'None'
    }
    
    is_updated = False
    priority = -1
    
    failure = (is_updated, data, priority)
    player_name = self.config['player_name']
    
    # check if LMS Query object is initiated
    if not hasattr(self, 'my_lms'):
        # add LMSQuery object to self
        logging.debug(f'building LMS Query object for player: {player_name}')
        self.my_lms = lmsquery.LMSQuery(player_name=player_name)
    try:
        # fetch the now playing data for the player
        now_playing = self.my_lms.now_playing()
        # remove the time key to make comparisions now_playing data updates easier in the Plugin class
        if 'time' in now_playing:
            now_playing.pop('time')
    except requests.exceptions.ConnectionError as e:
        logging.error(f'could not find player "{player_name}": {e}')
        return failure
    except KeyError as e:
        logging.warning(f'error getting now plyaing information for "{player_name}": KeyError {e}')
        return failure
    
    if now_playing:
        data = now_playing
    
    if now_playing['mode'] == 'play':
        priority = self.max_priority
        is_updated = True
    elif now_playing['mode'] == 'pause':
        priority = self.max_priority + 1
        is_updated = True
    else:
        logging.warning('setting priority to -1')
        priority = -1
        is_updated = True
    
    print(is_updated, data, priority)
    return is_updated, data, priority

In [76]:
slim_plugin(slim_test)

True {'id': 15075, 'title': 'Tom Sawyer', 'artist': 'Rush', 'coverid': '64757566', 'duration': 274.494, 'album_id': '1855', 'genre': 'No Genre', 'album': 'Moving Pictures', 'artwork_url': 'http://192.168.178.9:9000/music/64757566/cover.jpg', 'mode': 'play'} 0


(True,
 {'id': 15075,
  'title': 'Tom Sawyer',
  'artist': 'Rush',
  'coverid': '64757566',
  'duration': 274.494,
  'album_id': '1855',
  'genre': 'No Genre',
  'album': 'Moving Pictures',
  'artwork_url': 'http://192.168.178.9:9000/music/64757566/cover.jpg',
  'mode': 'play'},
 0)

In [34]:
spot_layout = {
        'title': {                       # text only block
            'image': None,               # do not expect an image
            'max_lines': 2,              # number of lines of text
            'width': 1,                  # 1/1 of the width - this stretches the entire width of the display
            'height': 2/3,               # 1/3 of the entire height
            'abs_coordinates': (0, 0),   # this block is the key block that all other blocks will be defined in terms of
            'hcenter': True,             # horizontally center text
            'vcenter': True,             # vertically center text 
            'relative': False,           # this block is not relative to any other. It has an ABSOLUTE position (0, 0)
            'font': '../fonts/Anton/Anton-Regular.ttf', # path to font file
            'font_size': None            # Calculate the font size because none was provided
        },
    
        'artist': {
            'image': None,
            'max_lines': 1,
            'width': 1,
            'height': 1/3,
            'abs_coordinates': [0, None],   # X = 0, Y will be calculated
            'hcenter': True,
            'vcenter': True,
            'font': '../fonts/Dosis/Dosis-VariableFontwght.ttf',
            'relative': ['artist', 'title'], # use the X postion from abs_coord from `artist` (this block: 0)
                                           # calculate the y position based on the size of `title` block
        }
}

spot_test = A()
spot_test.config = {'player_name': 'SpoCon-Spotify'}
spot_test.max_priority = 0

import requests
from json import JSONDecodeError
import constants_spot
from dictor import dictor
def spot_plugin(self):
  
    data = {
        'title': 'Err: no data',
        'artist': 'Err: no data',
        'album': 'Err: no data',
        'artwork_url': 'Err: no data',
        'duration': 0,
        'player': 'Err: no data',
        'mode': 'None'}
    priority = -1
    is_updated = False
    
    failure = (is_updated, data, priority)
    
    # attempt to get a token from librespot
    logging.debug(f'fetching access token from librespot player: {self.config["player_name"]}')
    logging.debug(f'scope: {constants_spot.spot_scope}, endpoint: {constants_spot.spot_player_endpoint}')
    try:
        token = requests.post(constants_spot.libre_token_url)
    except requests.ConnectionError as e:
        logging.error(f'Failed to pull Spotify token from librespot at url: {constants_spot.libre_token_url}')
        logging.error(f'{e}')
        return failure
    
    if token.status_code == 200:
        logging.debug('token received')
        try:
            headers = {'Authorization': 'Bearer ' + token.json()['token']}
        except JSONDecodeError as e:
            logging.error(f'failed to decode token JSON object: {e}')
            return failure
    else:
        logging.info(f'no token available from librespot status: {token.status_code}')
        return failure
    
    # use the token to fetch player information from spotify
    if 'Authorization' in headers:
        player_status = requests.get(constants_spot.spot_player_url, headers=headers)
    else:
        logging.warning(f'no valid Authroization token found in response from librespot: {headers}')
        return failure
    
    if player_status.status_code == 200:
        try:
            player_json = player_status.json()
        except JSONDecodeError as e:
            logging.error(f'failed to decode player status JSON object: {e}')
            return failure
        
        # bail out if the player name does not match
        if not dictor(player_json, 'device.name') == self.config['player_name']:
            logging.info(f'{self.config["player_name"]} is not active: no data')
            return failure
        
        for key in constants_spot.spot_map:
            data[key] = dictor(player_json, constants_spot.spot_map[key])
        
        playing = dictor(player_status.json(), 'is_playing')
        if playing is True:
            data['mode'] = 'play'
            is_updated = True
            priority = self.max_priority 
        elif playing is False:
            data['mode'] = 'paused'
            is_updated = True
            priority = self.max_priority + 1
        else:
            data['mode'] = None            
            is_updated = True
            priority = -1
  
    else:
        logging.info(f'no valid token available; request at {constants_spot.spot_player_url} resulted in status code: {player_status.status_code}')
        return failure
  
    
    return (is_updated, data, priority)
    # report the playing state
#     if playing == True:
#         logging.debug('playing')
#         return (True, data, self.max_priority)
#     else:
#         logging.debug('not playing')
#         return (False, data, -1)

In [35]:
spot_plugin(spot_test)

INFO:root:no valid token available; request at https://api.spotify.com/v1/me/player resulted in status code: 204


(False,
 {'title': 'Err: no data',
  'artist': 'Err: no data',
  'album': 'Err: no data',
  'artwork_url': 'Err: no data',
  'duration': 0,
  'player': 'Err: no data',
  'mode': 'None'},
 -1)

In [77]:
# plugin_digit_clock = Plugin(name='digit clock', 
#                             resolution=(500, 350), 
#                             layout=clock_layout, 
#                             update_function=clock_plugin,
#                             config={'update_interval': 10})
config = {
    'main': {
        'resolution': (500, 400)
    },
    'Plugin: Clock': {
        'layout': clock_layout,
        'update_function': clock_plugin,
        'update_rate': 30,
        'min_display_time': 10,
    },
    'Plugin: LMS': {
        'layout': slim_layout,
        'update_function': slim_plugin,
        'update_rate': 10,
        'player_name': 'MacPlay',
        'min_display_time': 20,
        
    },
#     'Plugin: LibreSpot': {
#         'layout': layout,
#         'update_function': spot_plugin,
#         'update_rate': 10,
#         'port': 24879, 
#         'host': 'localhost',
#         'min_display_time': 20,
#         'player_name': 'SpoCon-Spotify'
#     }
}

from IPython.display import display

plugins = {}
for section in config:
    if section.startswith('Plugin:'):
        logging.info(f'<<<<< building plugin: {section} >>>>>')
        my_plugin = Plugin(name=section,
                           resolution=config['main']['resolution'],
                           layout=config[section]['layout'],
                           update_function=config[section]['update_function'],
                           config=config[section], 
                           min_display_time=config[section]['min_display_time'],
                           refresh_rate=config[section]['update_rate'])
        # force the first update immediately
        my_plugin.update_function()
        
        plugins[section] = my_plugin


INFO:root:<<<<< building plugin: Plugin: Clock >>>>>
INFO:root:set text block: digit_time
INFO:root:TextBlock created
INFO:root:<<<<< building plugin: Plugin: LMS >>>>>
INFO:root:set text block: title
INFO:root:TextBlock created
INFO:root:set text block: artist
INFO:root:TextBlock created


True {'id': 15075, 'title': 'Tom Sawyer', 'artist': 'Rush', 'coverid': '64757566', 'duration': 274.494, 'album_id': '1855', 'genre': 'No Genre', 'album': 'Moving Pictures', 'artwork_url': 'http://192.168.178.9:9000/music/64757566/cover.jpg', 'mode': 'play'} -1


In [ ]:
logging.getLogger().setLevel('DEBUG')

In [78]:
plugins['Plugin: LMS'].update_function()

True {'id': 15075, 'title': 'Tom Sawyer', 'artist': 'Rush', 'coverid': '64757566', 'duration': 274.494, 'album_id': '1855', 'genre': 'No Genre', 'album': 'Moving Pictures', 'artwork_url': 'http://192.168.178.9:9000/music/64757566/cover.jpg', 'mode': 'play'} -1


(True,
 {'id': 15075,
  'title': 'Tom Sawyer',
  'artist': 'Rush',
  'coverid': '64757566',
  'duration': 274.494,
  'album_id': '1855',
  'genre': 'No Genre',
  'album': 'Moving Pictures',
  'artwork_url': 'http://192.168.178.9:9000/music/64757566/cover.jpg',
  'mode': 'play'},
 -1)

In [ ]:
import time
display_timer = Update()
min_display_time = 0
set_by = None

while True:
    for p in plugins:
        logging.info(f'[[ {p} ]]')
        logging.info(f'display timer: {display_timer.last_updated}')
        logging.info(f'{set_by} min_display_time: {min_display_time}')
        
        if plugins[p].poll() and display_timer.last_updated > min_display_time:
#         if display_timer.last_updated > min_display_time:
            display(plugins[p].image)
            display_timer.update()
            min_display_time = plugins[p].min_display_time
            set_by = plugins[p].name

    time.sleep(1)


In [ ]:
## Sample layout
# layout = {
#         'title': {                       # text only block
#             'image': None,               # do not expect an image
#             'max_lines': 2,              # number of lines of text
#             'width': 1,                  # 1/1 of the width - this stretches the entire width of the display
#             'height': 2/3,               # 1/3 of the entire height
#             'abs_coordinates': (0, 0),   # this block is the key block that all other blocks will be defined in terms of
#             'hcenter': True,             # horizontally center text
#             'vcenter': True,             # vertically center text 
#             'relative': False,           # this block is not relative to any other. It has an ABSOLUTE position (0, 0)
#             'font': '../fonts/Anton/Anton-Regular.ttf', # path to font file
#             'font_size': None            # Calculate the font size because none was provided
#         },
    
#         'artist': {
#             'image': None,
#             'max_lines': 1,
#             'width': 1,
#             'height': 1/3,
#             'abs_coordinates': [0, None],   # X = 0, Y will be calculated
#             'hcenter': True,
#             'vcenter': True,
#             'font': '../fonts/Dosis/Dosis-VariableFontwght.ttf',
#             'relative': ['artist', 'title'], # use the X postion from abs_coord from `artist` (this block: 0)
#                                            # calculate the y position based on the size of `title` block
#         }
# }

In [ ]:
# class Plugin:
#     '''base class for plugins'''
    
#     def __init__(self, name=None, resolution=None, layout=None, 
#                  max_priority=-1, update_function=None, update_rate=10,
#                  min_display_time=10, config={}):
#         self.name = name
#         self.timer = Update()
#         self.last_ask = 0
#         self.update_rate = update_rate
#         self.resolution = resolution
#         self.max_priority = max_priority
#         self.priority = -1
#         self.min_display_time = min_display_time
#         self.config = config
#         self.data = {} # init the property
        
#         if resolution:
#             self.layout = layout
#         self.add_update_function(update_function)
    
    
#     def add_update_function(self, function):
#         '''add an update_function to the namespace of this instance of object
#             all update_functions must accept at a minimum the following kwargs:
#                 * config(`dict`) configuration dictionary passed to all update functions
#             all update_functions must return a tuple of:
#                 (update, data, priority)
                
#                 update(`bool`): True, data has been updated; display requires update
#                 data(`dict`): structured for a Layout().update 
#                 priority(`int` or `float`): priority for inclusion in display loop
#                     positive values that approach 0 have higher priority

#         Args:
#             fuction(`function`): function to add as self.update_function
            
#         see https://stackoverflow.com/a/2982/5530152 regarding descriptor protocol''' 
#         if function:
#             self.update_function = function.__get__(self)
#         else:
#             self.update_function = self._null_function.__get__(self)
#     @property
#     def update_rate(self):
#         '''maximum rate in seconds update_function should be called
#             update_rate(`int` or `float`): N'''
#         return self._update_rate
    
#     @update_rate.setter
#     def update_rate(self, update_rate):
#         if not isinstance(update_rate, (int, float)):
#             raise TypeError('update_rate must be type int or float')
#         if not update_rate > 0:
#             raise ValueError('update_rate must be > 0')
#         self._update_rate = update_rate
    
#     @property 
#     def data(self):
#         '''data returned from self.update_function
#         data(`dict`): {'key': 'value'}'''
#         return self._data
    
#     @data.setter
#     def data(self, data):
#         self._data = data
    
#     @property
#     def min_display_time(self):
#         '''minimum time in seconds to display this plugin screen -- default is 10s
        
#         time(`int` or `float`): N'''
#         return self._min_display_time
    
#     @min_display_time.setter
#     def min_display_time(self, time):
#         if not isinstance(time, (int, float)):
#             raise TypeError('time must be of type int or float')
#         if not time > 0:
#             raise ValueError('time must be greater than 0')
    
#         self._min_display_time = time
        
    
#     @property
#     def name(self):
#         '''name of plugin derived from configuration file -- always in the format: `Plugin: XXXXX`
#         name(`str`): Plugin: Name'''
#         return self._name

#     @name.setter
#     def name(self, name):
#         self._name = str(name)
        

# #     @property
# #     def last_update(self):
# #         '''display the monotnic time since this module was updated'''
# #         return self.timer.last_updated
    
#     @property
#     def max_priority(self):
#         '''maximum priority for displaying this module in the update loop
#             positive values that approach zero are higher priority; zero is highest
#             negative values are excluded entirely from update loop
        
#         max_priority(`int` or `float`)
#         '''
#         return self._max_priority
    
#     @max_priority.setter
#     @strict_enforce(int, float)
#     def max_priority(self, max_priority):
#         self._max_priority = max_priority
    
#     @property
#     def image(self):
#         '''constructed image based on the layout'''
#         return self.layout_obj.concat()
    
#     @property
#     def resolution(self):
#         '''width x height of screen
#         resolution(`tuple` of `int`): (W x H)'''
#         return self._resolution
    
#     @resolution.setter
#     def resolution(self, resolution):
#         if isinstance(resolution, (type(None), list, tuple)):
#             self._resolution = resolution
#             if hasattr(self, 'layout_obj'):
#                 # trigger recalculation of the layout if the resolution changes
#                 layout = self.layout
#                 self.layout = layout
#         else:
#             raise ValueError('`resolution` must be a list-like object')
            
    
#     @property
#     def layout(self):
#         '''layout dictionary for placing text and images blocks on the screen'''
#         return self.layout_obj.layout
    
#     @layout.setter
#     def layout(self, layout):
#         if self.resolution:
#             self.layout_obj = Layout(resolution=self.resolution, layout=layout) 
#         else:
#             raise ValueError(f'cannot set `layout` without a valid resolution: {self.resolution}')


#     def check(self):
#         if time.monotonic() - self.last_ask > self.update_rate:
#             self.last_ask = time.monotonic()
#             return True
#         else:
#             logging.debug(f'wait for {self.update_rate - (time.monotonic - self.last_ask)} seconds')
#             return False
            
    
#     def _null_function(self, *args, **kwargs):
#         logging.info('default placeholder function')
#         return (False, {}, -1)
                

#     def Xpoll(self, *args, **kwargs):
#         # this could benefit from some throttling 
#         # once last_updated > update_rate this gets hammered with update requests every refresh cycle
        
#         # a rate limiter decorator would be perfect here -- need to figure out how to pass self.interval
#         # to the rate limiter
        
#         if self.timer.last_updated >= self.update_rate:
#             logging.debug('last_update > update_rate -- checking for new data')
#             is_updated, data, priority = self.update_function(*args, **kwargs)
#             logging.debug(f'is_updated: {is_updated}, data: {data}, priority: {priority}')
#         else:
#             logging.debug(f'{self.timer.last_updated} < {self.update_rate} -- not checking for new data')
#             is_updated = False
            
#         if is_updated:
#             logging.debug('update_function reports updated data')
#             self.priority = priority
            
#             if data == self.data:
#                 logging.debug('new data matches old data; no image refresh needed')
#                 ret_val = True
#                 # ugly hack to rate limit: reset the update clock
#                 self.timer.update()
                
#             else:
#                 logging.debug('new data does not match old data; refreshing image')
#                 self.layout_obj.update_contents(data)
#                 self.timer.update()
#                 self.data = data
#                 ret_val = True
        
#         else:
#             ret_val = False
            
#         return ret_val